# Passo necessários para configurar o spark

In [1]:
# Para que o Jupyter consiga carregar o Spark corretamente no notebook
import findspark
findspark.init('/usr/local/spark')
# findspark.init('/usr/lib/spark')

# Para que os executors tenham mais memória e não falhem por falta de recursos
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--executor-memory 1G pyspark-shell'

# A partir daqui é código Spark que normalmente é executado com um comando similar ao comando abaixo:
# spark-submit --executor-memory 1G nome_do_script.py
from pyspark import SparkConf, SparkContext

# A linha abaixo está comentada porque essa é a forma de executar Spark em uma instalação local usando todos os cores
# conf = SparkConf().setMaster("local[*]").setAppName("NomeDoApp")

conf = SparkConf()
sc = SparkContext(conf=conf)

# Importando as libs necessárias para o projeto

In [2]:
import pandas as pd
from s3fs import S3FileSystem

s3 = S3FileSystem(anon=True)

# Carregando datasets

In [3]:
file_name = 'data-eng-t2-school/projetos/problema-06/dados/problema-06-sample-01.csv'

with s3.open(file_name, 'rb') as f:
    df_visits = pd.read_csv(f)

In [ ]:
file_name = 'data-eng-t2-school/projetos/problema-06/dados/problema-06_estabelecimentos.csv'

with s3.open(file_name, 'rb') as f:
    df_stores = pd.read_csv(f)

In [ ]:
df_stores.info()

# Preprocessamento dos datasets

In [ ]:
df_stores['category'] = df_stores['category'].apply(lambda x: x.strip())

In [ ]:
df_visits['weekday'] = pd.to_datetime(df_visits['timestamp']).map(lambda x: x.dayofweek)

In [ ]:
result = pd.merge(df_visits, df_stores, how='inner', on=['lat', 'lng'])

In [ ]:
count_series = result[['user_id']].groupby(['user_id']).size()
# df.nlargest(3, 'a')
new_df = count_series.to_frame(name = 'size').reset_index()
top_users = new_df.nlargest(10, 'size')
type(top_users)

In [ ]:
top_results = result[result.user_id.isin(top_users.user_id)]

In [ ]:
grouped_results = top_results[['user_id', 'category', 'weekday']].groupby(['user_id', 'category', 'weekday']).size()

In [ ]:
grouped_results.values
with pd.option_context('display.max_rows', 1000, 'display.max_columns', 3):
    print(grouped_results)
type(grouped_results)

In [ ]:
# Import pyplot
import matplotlib.pyplot as plt

grouped_results.plot()
plt.show()